In [7]:
import pandas as pd
import pathlib
from plotly import graph_objects as go

from pandora.embedding import from_smartpca
from pandora.plotting import plot_populations, _update_fig, _get_hovertemplate

RESULTS = pathlib.Path("results")
PLOTS_DIR = RESULTS / "plots"

C0 = "rgb(0, 150, 130)"

# HO-WE-Çayönü Analyses

In [8]:
def plt(df):
    hover_cols = ["sample_id", "population"]
    hover_cols_display = ["Sample", "Population"]

    fig = go.Figure(
        go.Scatter(
            x=df.embedding.D0,
            y=df.embedding.D1,
            mode="markers",
            marker_color="lightgray",
            customdata=df.embedding[hover_cols].values,
            hovertemplate=_get_hovertemplate(hover_cols_display),
        )
    )

    low_svs = df.embedding.loc[lambda x: x.sample_id == "cay015"]


    fig.add_trace(
        go.Scatter(
            x=low_svs.D0,
            y=low_svs.D1,
            mode="markers+text",
            marker_color=C0,
            marker_symbol="star",
            marker_size=12,
            text=["cay015"],
            textposition=["bottom center"],
            customdata=df.embedding[["sample_id", "population"]].values,
        )
    )

    _update_fig(df, fig, 0, 1, show_variance_in_axes=True)
    fig.update_layout(height=750, width=750, showlegend=False)
    return fig

CAYONU_RESULTS = RESULTS / "HO-Cayonu"

In [9]:

modern_list = CAYONU_RESULTS / "modern.poplist.txt"
modern_populations = pd.Series([pop.strip() for pop in modern_list.open()])

pca = from_smartpca(
    CAYONU_RESULTS / "HO-Cayonu.evec",
    CAYONU_RESULTS / "HO-Cayonu.eval"
)
plt(pca)

In [10]:
# idx = 3  # Libyan-Jew
idx = 11  # Yemenite-Jew
evec = CAYONU_RESULTS  / f"bootstrap_{idx}.evec"
eval = CAYONU_RESULTS / f"bootstrap_{idx}.eval"

pca = from_smartpca(evec, eval)
pca.embedding["D0"] = -pca.embedding["D0"]
if idx == 3:
    pca.embedding["D1"] = -pca.embedding["D1"]

fig = plt(pca)
replicate_no = 1 if idx == 11 else 2
fig.update_layout(title=f"Bootstrap Replicate #{replicate_no}")
fig.write_image(PLOTS_DIR / f"bootstrap_{replicate_no}.pdf")
fig